In [1]:
!pip install --upgrade git+https://github.com/flairNLP/flair.git

  Cloning https://github.com/flairNLP/flair.git to /tmp/pip-req-build-stlm912b
  Running command git clone -q https://github.com/flairNLP/flair.git /tmp/pip-req-build-stlm912b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 256kB 10.1MB/s 
     |████████████████████████████████| 983kB 14.0MB/s 
     |████████████████████████████████| 645kB 27.3MB/s 
     |████████████████████████████████| 798kB 16.6MB/s 
     |████████████████████████████████| 1.1MB 45.3MB/s 
     |████████████████████████████████| 890kB 53.3MB/s 
     |████████████████████████████████| 3.8MB 54.4MB/s 
  Created wheel for flair: filename=flair-0.4.5-cp36-none-any.whl size=148505 sha256=6c948c0e64a7dd3183fb4a438906803bec39800a9a0d13c51e86ae2e749e9dbf
  Stored in directory: /tmp/pip-ephem-wheel-cache-56ucurov/wheels/84/82/73/d2b3b59b7be74ea05f2c6d64132efe27df52daffb47d1dc7bb
Successfully built flair
  Cre

In [0]:
from flair.data_fetcher import NLPTaskDataFetcher
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentLSTMEmbeddings, DocumentRNNEmbeddings, BertEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
from pathlib import Path
from flair.datasets import CSVClassificationCorpus
from flair.data import Corpus
import pandas as pd

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
data_folder = "/content/drive/My Drive/Capstone/new_data/"

### First Stage (Train on benchmark dataset)

In [0]:
benchmark = pd.read_csv(data_folder + "combined_benchmark.csv")

In [6]:
benchmark = benchmark[['label', 'text']]
benchmark.head()

,label,text
0,0,Why not subscribe to the magazine ?
1,-1,"Tornio Works employs 2,300 of whom more than 1..."
2,1,"The move is aimed at boosting sales , cost-eff..."
3,0,"As a result of the merger , the largest profes..."
4,-1,18 March 2010 A leakage in the gypsum pond was...


#### Create train, dev and test set

In [0]:
# benchmark = benchmark.sample(frac=1)
# benchmark.iloc[0:int(len(benchmark)*0.8)].to_csv(data_folder + 'train.csv', sep='\t', index = False, header = False)
# benchmark.iloc[int(len(benchmark)*0.8):int(len(benchmark)*0.9)].to_csv(data_folder + 'test.csv', sep='\t', index = False, header = False)
# benchmark.iloc[int(len(benchmark)*0.9):].to_csv(data_folder + 'dev.csv', sep='\t', index = False, header = False)

In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split
import sklearn

In [0]:
kaggle_X_train, kaggle_X_te, kaggle_Y_train, kaggle_Y_te = train_test_split(benchmark['text'],benchmark['label'], test_size = 0.2, random_state = 0, stratify = benchmark['label'])
kaggle_X_dev, kaggle_X_test, kaggle_Y_dev, kaggle_Y_test = train_test_split(kaggle_X_te,kaggle_Y_te, test_size = 0.5, random_state = 0)

In [0]:
benchmark_train_df =  pd.DataFrame({ 'label': kaggle_Y_train, 'text': kaggle_X_train   })
benchmark_dev_df = pd.DataFrame({ 'label': kaggle_Y_dev , 'text': kaggle_X_dev  })
benchmark_test_df = pd.DataFrame({ 'label': kaggle_Y_test, 'text': kaggle_X_test   })

In [0]:
benchmark_train_df.to_csv( data_folder + "train.csv",  sep='\t', index = False, header = False)
benchmark_dev_df.to_csv( data_folder + "dev.csv",sep='\t', index = False, header = False)
benchmark_test_df.to_csv( data_folder + "test.csv", sep='\t',index = False, header = False)

#### Build corpus

In [11]:
# corpus = NLPTaskDataFetcher.load_classification_corpus(Path(data_folder), test_file='test.csv', dev_file='dev.csv', train_file='train.csv')
column_name_map = {1: "text", 0: "label_topic"}

corpus: Corpus = CSVClassificationCorpus(data_folder,
                                         column_name_map,
                                         skip_header=False, #no header in kaggle data
                                         delimiter='\t',    # comma separated rows
)

2020-05-22 00:41:15,766 Reading data from /content/drive/My Drive/Capstone/new_data
2020-05-22 00:41:15,767 Train: /content/drive/My Drive/Capstone/new_data/train.csv
2020-05-22 00:41:15,768 Dev: /content/drive/My Drive/Capstone/new_data/dev.csv
2020-05-22 00:41:15,769 Test: /content/drive/My Drive/Capstone/new_data/test.csv


#### Create word embeddings

In [12]:
word_embeddings = [BertEmbeddings(), FlairEmbeddings('news-forward-fast'), FlairEmbeddings('news-backward-fast')]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated method __init__. (Use 'TransformerWordEmbeddings' for all transformer-based word embeddings) -- Deprecated since version 0.4.5.
  """Entry point for launching an IPython kernel.



2020-05-22 00:41:53,809 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings/lm-news-english-forward-1024-v0.2rc.pt not found in cache, downloading to /tmp/tmpcfliebv5


100%|██████████| 19689779/19689779 [00:02<00:00, 9825041.73B/s]

2020-05-22 00:41:56,565 copying /tmp/tmpcfliebv5 to cache at /root/.flair/embeddings/lm-news-english-forward-1024-v0.2rc.pt
2020-05-22 00:41:56,586 removing temp file /tmp/tmpcfliebv5


2020-05-22 00:42:13,684 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings/lm-news-english-backward-1024-v0.2rc.pt not found in cache, downloading to /tmp/tmpm9k20d0s


100%|██████████| 19689779/19689779 [00:02<00:00, 9182361.40B/s]

2020-05-22 00:42:16,524 copying /tmp/tmpm9k20d0s to cache at /root/.flair/embeddings/lm-news-english-backward-1024-v0.2rc.pt
2020-05-22 00:42:16,543 removing temp file /tmp/tmpm9k20d0s


#### First Stage Fine-tuning

In [13]:
document_embeddings = DocumentLSTMEmbeddings(word_embeddings, hidden_size=512, reproject_words=True, reproject_words_dimension=256)
classifier = TextClassifier(document_embeddings, label_dictionary=corpus.make_label_dictionary(), multi_label=False)
trainer = ModelTrainer(classifier, corpus)
trainer.train(data_folder, max_epochs=10)

2020-05-22 00:42:29,759 Computing label dictionary. Progress:


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated method __init__. (The functionality of this class is moved to 'DocumentRNNEmbeddings') -- Deprecated since version 0.4.
  """Entry point for launching an IPython kernel.
100%|██████████| 1315/1315 [00:01<00:00, 1014.58it/s]

2020-05-22 00:42:31,312 [b'1', b'0', b'-1']
2020-05-22 00:42:31,325 ----------------------------------------------------------------------------------------------------
2020-05-22 00:42:31,329 Model: "TextClassifier(
  (document_embeddings): DocumentLSTMEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): BertEmbeddings(
        (model): BertModel(
          (embeddings): BertEmbeddings(
            (word_embeddings): Embedding(30522, 768, padding_idx=0)
            (position_embeddings): Embedding(512, 768)
            (token_type_embeddings): Embedding(2, 768)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (encoder): BertEncoder(
            (layer): ModuleList(
              (0): BertLayer(
                (attention): BertAttention(
                  (self): BertSelfAttention(
                    (query): Linear(in_features=768, out_features=768, bias=Tru

2020-05-22 00:42:33,340 epoch 1 - iter 3/37 - loss 1.41591505 - samples/sec: 56.25
2020-05-22 00:42:45,293 epoch 1 - iter 6/37 - loss 1.45490547 - samples/sec: 67.57
2020-05-22 00:42:57,051 epoch 1 - iter 9/37 - loss 1.31488357 - samples/sec: 68.15
2020-05-22 00:43:08,577 epoch 1 - iter 12/37 - loss 1.28688369 - samples/sec: 80.23
2020-05-22 00:43:20,193 epoch 1 - iter 15/37 - loss 1.23950537 - samples/sec: 82.39
2020-05-22 00:43:31,961 epoch 1 - iter 18/37 - loss 1.27530628 - samples/sec: 70.68
2020-05-22 00:43:43,390 epoch 1 - iter 21/37 - loss 1.28792808 - samples/sec: 73.44
2020-05-22 00:43:55,197 epoch 1 - iter 24/37 - loss 1.26882177 - samples/sec: 75.74
2020-05-22 00:44:06,782 epoch 1 - iter 27/37 - loss 1.25488860 - samples/sec: 82.36
2020-05-22 00:44:18,182 epoch 1 - iter 30/37 - loss 1.23522135 - samples/sec: 75.24
2020-05-22 00:44:29,490 epoch 1 - iter 33/37 - loss 1.22724476 - samples/sec: 87.06
2020-05-22 00:44:40,844 epoch 1 - iter 36/37 - loss 1.21286295 - samples/sec: 8

{'dev_loss_history': [0.8998611569404602,
  0.9690008163452148,
  1.2271349430084229,
  1.2768433094024658,
  0.7436613440513611,
  0.6968156695365906,
  0.9381842017173767,
  0.8523504137992859,
  0.7248088717460632,
  0.7704737782478333],
 'dev_score_history': [0.7397260273972602,
  0.6986301369863014,
  0.5799086757990868,
  0.5753424657534246,
  0.7397260273972602,
  0.7534246575342466,
  0.730593607305936,
  0.7534246575342466,
  0.7579908675799086,
  0.8036529680365296],
 'test_score': 0.7732426303854876,
 'train_loss_history': [1.2048546929617185,
  0.9636928213609232,
  0.8570267461441659,
  0.7829802527620986,
  0.7690218107120411,
  0.7448374612911327,
  0.6764946853792345,
  0.6988622038751036,
  0.6586972636145514,
  0.5799998939037323]}

### Second Stage (train on hand annotated datasets)

#### Build corpus

In [0]:
new_data_folder = '/content/drive/My Drive/Capstone/second_stage_data/'

#### Second Stage fine-tuning

In [0]:
# benchmark_embeddings = FlairEmbeddings(data_folder + 'best-model.pt')
benchmark_classifier = TextClassifier.load(data_folder + 'best-model.pt')

2020-05-21 22:56:22,462 loading file ./drive/My Drive/capstone/data/best-model.pt


In [0]:
trainer = ModelTrainer(benchmark_classifier, corpus)
trainer.train(new_data_folder, max_epochs=10)

2020-05-21 22:56:25,413 ----------------------------------------------------------------------------------------------------
2020-05-21 22:56:25,418 Model: "TextClassifier(
  (document_embeddings): DocumentLSTMEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): BertEmbeddings(
        (model): BertModel(
          (embeddings): BertEmbeddings(
            (word_embeddings): Embedding(30522, 768, padding_idx=0)
            (position_embeddings): Embedding(512, 768)
            (token_type_embeddings): Embedding(2, 768)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (encoder): BertEncoder(
            (layer): ModuleList(
              (0): BertLayer(
                (attention): BertAttention(
                  (self): BertSelfAttention(
                    (query): Linear(in_features=768, out_features=768, bias=True)
                    (key): Linear(in_feat

{'dev_loss_history': [0.6211347579956055,
  0.6653227806091309,
  0.5546189546585083,
  0.4246145486831665,
  0.34248945116996765,
  2.3637852668762207,
  0.3363116681575775,
  1.2868273258209229,
  0.8674789667129517,
  0.4017622470855713],
 'dev_score_history': [0.7777777777777778,
  0.7777777777777778,
  0.7777777777777778,
  0.8383838383838383,
  0.8787878787878788,
  0.6161616161616161,
  0.8787878787878788,
  0.7171717171717171,
  0.8181818181818182,
  0.898989898989899],
 'test_score': 0.898989898989899,
 'train_loss_history': [1.22447270154953,
  1.0648577809333801,
  0.7276832222938537,
  0.6568765461444854,
  0.4097093641757965,
  0.4592723071575165,
  0.9268317580223083,
  0.26085380017757415,
  0.3351425021886826,
  0.37118481993675234]}

In [17]:
final_classifier = TextClassifier.load(new_data_folder + 'final-model.pt')

2020-05-22 02:10:33,883 loading file /content/drive/My Drive/Capstone/second_stage_data/final-model.pt


In [19]:
final_classifier.predict("Canada's economy shrinks in October from auto strike spillover. Canada’s economy contracted in October for the first time in eight months, as the United Auto Workers strike in the U.S. weighed on plant production")

[Sentence: "Canada's economy shrinks in October from auto strike spillover. Canada’s economy contracted in October for the first time in eight months, as the United Auto Workers strike in the U.S. weighed on plant production"   [− Tokens: 34  − Sentence-Labels: {'class': [-1 (0.9992)]}]]